In [39]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
import pickle
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import logging
from open_spiel.python.examples.ubc_mccfr_cpp_example import action_to_bids
from open_spiel.python.examples.ubc_nfsp_example import policy_from_checkpoint
from open_spiel.python.examples.ubc_utils import *

from open_spiel.python.pytorch.ubc_nfsp import NFSP
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap

output_notebook()
# logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

In [40]:
EXPERIMENT_DIR = '/shared/outputs/big_game_2/lstm_deep'

In [41]:
CHECKPOINT_NAME = 'checkpoint_latest'
checkpoint = policy_from_checkpoint(EXPERIMENT_DIR, CHECKPOINT_NAME)

In [42]:
def check_on_q_values(checkpoint):
    q_network = checkpoint.agents[0]._rl_agent._q_network
    state = checkpoint.game.new_initial_state().child(0).child(0)
    legal_actions = state.legal_actions()
    it = state.information_state_tensor()
    info_state = q_network.prep_batch([q_network.reshape_infostate(it)])
    q_values = q_network(info_state).detach()[0]
    legal_q_values = q_values[legal_actions]
    action_dict = get_actions(checkpoint.game)
    return {s: q for s,q in zip(action_dict.values(), legal_q_values)}

In [38]:
actions = []
for episode in tqdm(range(1000000)):
    if episode % 50000 == 0:
        qs = check_on_q_values(checkpoint)
        print("Bid for 1,0,0 licenses @ $75 with activity 75 -", qs["Bid for 1,0,0 licenses @ $75 with activity 75"])
        print("Bid for 1,0,1 licenses @ $100 with activity 100 -", qs["Bid for 1,0,1 licenses @ $100 with activity 100"])
        print(pd.Series(actions).value_counts())
        
    time_step = checkpoint.env.reset()
    first_action = True

    while not time_step.last():
        player_id = time_step.observations["current_player"]
        agent = checkpoint.agents[player_id]
        # Player 1 is fixed
        agent_output = agent.step(time_step, is_evaluation = player_id==1) 
#         if player_id == 0 and agent_output.action != 4:
#             print(agent_output.action)
        
        if player_id == 0 and first_action:
            actions.append(agent_output.action)
            first_action = False
        action_list = [agent_output.action]
        time_step = checkpoint.env.step(action_list)

    # Episode is over, step all agents with final info state.
    for player_id, agent in enumerate(checkpoint.agents):
        agent.step(time_step)
    

  0%|          | 0/1000000 [00:00<?, ?it/s]/apps/open_spiel/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
  0%|          | 17/1000000 [00:00<1:42:08, 163.16it/s]

Bid for 1,0,0 licenses @ $75 with activity 75 - tensor(44.9937)
Bid for 1,0,1 licenses @ $100 with activity 100 - tensor(15.0661)
Series([], dtype: int64)


  5%|▌         | 50018/1000000 [06:01<2:07:53, 123.80it/s]

Bid for 1,0,0 licenses @ $75 with activity 75 - tensor(45.0075)
Bid for 1,0,1 licenses @ $100 with activity 100 - tensor(15.0917)
4     49544
0       382
1        36
3         8
6         6
10        5
9         4
8         4
5         4
2         3
11        2
7         2
dtype: int64


 10%|█         | 100015/1000000 [12:07<2:03:36, 121.35it/s]

...

 95%|█████████▍| 949989/1000000 [1:53:32<05:46, 144.31it/s]

Bid for 1,0,0 licenses @ $75 with activity 75 - tensor(45.0122)
Bid for 1,0,1 licenses @ $100 with activity 100 - tensor(15.4791)


 95%|█████████▌| 950026/1000000 [1:53:32<07:10, 116.04it/s]

4     940230
0       8262
1        669
5        102
9        100
7         98
10        93
3         93
8         90
6         89
2         89
11        85
dtype: int64


100%|██████████| 1000000/1000000 [1:59:36<00:00, 139.35it/s]


In [ ]:
[d for d in checkpoint.agents[0]._rl_agent._replay_buffer._data if d.reward in [50., 30.]]

In [ ]:
pd.DataFrame(q).drop_duplicates()

In [ ]:
df = pd.DataFrame.from_records(records)

In [ ]:
df = df.drop_duplicates(subset=['infostate_string'])
print(f'After dropping, {len(df)} unique infostates remaining')